In [8]:
import pandas as pd
from datetime import datetime
import math
from pymongo import MongoClient

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def analyze_clicks_gaze(gaze_data, click_data):
    total_count = 0

    for index,click in click_data.iterrows():
        click_time = click['normalizedTimestamp']
        click_x = click['x']
        click_y = click['y']

        min_timestamp = click_time - 2000  # 1 second before click in milliseconds
        max_timestamp = click_time + 2000  # 1 second after click in milliseconds

        user_gaze = [g for i,g in gaze_data.iterrows() if
                     g["normalizedTimestamp"] >= min_timestamp and
                     g["normalizedTimestamp"] <= max_timestamp
                     and             
                     calculate_distance(g["x"], g["y"], click_x, click_y) <= 100]

        if user_gaze:
            total_count += 1
    
    return total_count

def main():
    # Connect to MongoDB
    client = MongoClient("mongodb://localhost:27017")
    db = client["INTUX"]
    
    # Assuming "gaze_collection" and "click_collection" are your MongoDB collection names
    gaze_data = list(db["GazeData"].find())
    click_data = list(db["ClickData"].find())
    meta_data = list(db["MetaData"].find())

    gaze_df = pd.DataFrame(gaze_data)
    click_df = pd.DataFrame(click_data)
    meta_df = pd.DataFrame(meta_data)

    gaze_df = gaze_df.drop(['_id'],axis=1)
    click_df = click_df.drop(['_id'],axis=1)
    meta_df = meta_df.drop(['_id'],axis=1)

    user_click_counts = {}

    for _, gaze_entry in meta_df.iterrows():
        roll_no = gaze_entry["rollNo"]
        if roll_no not in user_click_counts:
            user_click_counts[roll_no] = {"total_clicks": 0, "clicks_when_looking": 0}

    for roll_no, click_count in user_click_counts.items():
        selected_row = meta_df[meta_df['rollNo'] == roll_no]
        click_count["total_clicks"] = selected_row['clickCount'].values[0]

        gaze_roll = gaze_df[gaze_df['rollNo'] == roll_no]
        click_roll = click_df[click_df['rollNo'] == roll_no]

        click_count["clicks_when_looking"] = analyze_clicks_gaze(gaze_roll,click_roll)

    hit_click = 0
    total_click = 0

    yr1_total = 0
    yr2_total = 0
    yr3_total = 0
    yr4_total = 0

    yr1_click = 0
    yr2_click = 0
    yr3_click = 0
    yr4_click = 0


    for roll_no, click_count in user_click_counts.items():
        hit_click+=click_count['clicks_when_looking']
        total_click+=click_count['total_clicks']

        if roll_no[-4] == "3":
            yr1_total += click_count['total_clicks']
            yr1_click += click_count['clicks_when_looking']
        elif roll_no[-4] == "2":
            yr2_total += click_count['total_clicks']
            yr2_click += click_count['clicks_when_looking']
        elif roll_no[-4] == "1":
            yr3_total += click_count['total_clicks']
            yr3_click += click_count['clicks_when_looking']
        elif roll_no[-4] == "0":
            yr4_total += click_count['total_clicks']
            yr4_click += click_count['clicks_when_looking']

        print(f"User {roll_no}: {click_count['clicks_when_looking']} clicks out of {click_count['total_clicks']} were clicked when the user was looking at it.")

    print("Hit ratio",hit_click,total_click,hit_click/total_click)

    print("1st year data",yr1_click,yr1_total,yr1_click/yr1_total)
    print("2nd year data",yr2_click,yr2_total,yr2_click/yr2_total)
    print("3rd year data",yr3_click,yr3_total,yr3_click/yr3_total)
    print("4rth year data",yr4_click,yr4_total,yr4_click/yr4_total)


if __name__ == "__main__":
    main()


User CB.EN.U4CSE20402: 19 clicks out of 20 were clicked when the user was looking at it.
User CB.EN.U4CSE20403: 20 clicks out of 30 were clicked when the user was looking at it.
User CB.EN.U4CSE20421: 11 clicks out of 11 were clicked when the user was looking at it.
User CB.EN.U4CSE20425: 18 clicks out of 21 were clicked when the user was looking at it.
User CB.EN.U4CSE20429: 4 clicks out of 12 were clicked when the user was looking at it.
User CB.EN.U4CSE20431: 15 clicks out of 19 were clicked when the user was looking at it.
User CB.EN.U4CSE20435: 15 clicks out of 16 were clicked when the user was looking at it.
User CB.EN.U4CSE20436: 9 clicks out of 16 were clicked when the user was looking at it.
User CB.EN.U4CSE20447: 9 clicks out of 12 were clicked when the user was looking at it.
User CB.EN.U4CSE20449: 12 clicks out of 18 were clicked when the user was looking at it.
User CB.EN.U4CSE21002: 8 clicks out of 14 were clicked when the user was looking at it.
User CB.EN.U4CSE21014: 6 